## 🥸 CorpusLLM
*Summarizing & chunking*

## ⬇️ Imports

In [ ]:
from google import auth as google_auth
from google.colab import auth
import gspread
import pandas as pd
import numpy as np
import requests
import re

# 🤖 Prepare the LLM

In [ ]:
!pip install -q openai langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.0/817.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 12.6 MB/s eta 0:00:00


In [ ]:
# from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from google.colab import userdata

# Check if these are all necessary
from typing import List

from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

### OpenAI

In [ ]:
# modelName: 'gpt-4-1106-preview'
# modelName: 'gpt-3.5-turbo-1106'
openai_llm_4 = ChatOpenAI(temperature=0.4, openai_api_key=userdata.get('OPENAI_API_KEY'), model_name='gpt-4-1106-preview')
openai_llm_3 = ChatOpenAI(temperature=0.25, openai_api_key=userdata.get('OPENAI_API_KEY'), model_name='gpt-3.5-turbo-1106')

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


### OpenRouter (custom)
- https://medium.com/@gal.peretz/openrouter-langchain-leverage-opensource-models-without-the-ops-hassle-9ffbf0016da7

In [ ]:
# from langchain_community.chat_models import ChatOpenAI
from typing import Optional

class ChatOpenRouter(ChatOpenAI):
    openai_api_base: str
    openai_api_key: str
    model_name: str

    def __init__(self,
                 model_name: str,
                 openai_api_key: Optional[str] = None,
                 openai_api_base: str = "https://openrouter.ai/api/v1",
                 **kwargs):
        openai_api_key = userdata.get('OPENROUTER_API_KEY')
        super().__init__(openai_api_base=openai_api_base,
                         openai_api_key=openai_api_key,
                         model_name=model_name,
                         temperature=0.2, # Dirty
                         **kwargs)

In [ ]:
# openrouter_llm = ChatOpenRouter(model_name="cognitivecomputations/dolphin-mixtral-8x7b", temperature=0.2)
openrouter_llm = ChatOpenRouter(model_name="cognitivecomputations/dolphin-mixtral-8x7b")

### Test LLM models

In [ ]:
joke_prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")

openrouter_chain = joke_prompt | openrouter_llm
print(openrouter_chain.invoke({"topic": "banana"}))

openai_chain = joke_prompt | openai_llm_3
print(openai_chain.invoke({"topic": "banana"}))

content=" Why did the banana go to the doctor? Because it wasn't peeling well."
content="Why did the banana go to the doctor? Because it wasn't peeling well!"


### Summarize chapter chain

In [ ]:
summarize_chapter_prompt = ChatPromptTemplate.from_template("""
### CONTEXT AND QUESTION:
- You are a professional chapter summarizer
- You will recieve a CHAPTER
- MAKE SURE YOU SUMMARIZE IT IN THE SAME TONE as AND FROM THE SAME PERSPECTIVE as the chapter
- DO NOT MENTION THE AUTHOR'S NAME
- The response should be 1 short paragraph MAX 250 CHARACTERS
- RESPOND IN FIRST PERSON WRITING STYLE
- KEEP IT SHORT!
- KEEP IT SHORT! AND I WILL GIVE YOU $20 TIP!
- KEEP IT SHORT! MAX 100 CHARACTERS!
- KEEP IT SHORT! MAX 40 WORDS!

### CHAPTER:

{chapter}

### CONTEXT AND QUESTION:
- You are a professional chapter summarizer
- You will recieve a CHAPTER
- MAKE SURE YOU SUMMARIZE IT IN THE SAME TONE as AND FROM THE SAME PERSPECTIVE as the chapter
- DO NOT MENTION THE AUTHOR'S NAME
- The response should be 1 short paragraph MAX 250 CHARACTERS
- RESPOND IN FIRST PERSON WRITING STYLE
- KEEP IT SHORT!
- KEEP IT SHORT! AND I WILL GIVE YOU $20 TIP!
- KEEP IT SHORT! MAX 100 CHARACTERS!
- KEEP IT SHORT! MAX 40 WORDS!
- DO NOT USE WORDS SUCH AS NARRATIVE, AUTHOR OR NIETZSCHE
""")

# summarize_chapter_chain = summarize_result_prompt | openai_llm_4
summarize_chapter_chain = summarize_chapter_prompt | openrouter_llm

def summarize_chapter(chapter):
    response = summarize_chapter_chain.invoke({ 'chapter': chapter })
    return response.content

In [ ]:
summarize_chapter("""
I now wish to relate the history of _Zarathustra._ The fundamental
idea of the work, the _Eternal Recurrence,_ the highest formula of
a Yea-saying to life that can ever be attained, was first conceived
in the month of August 1881. I made a note of the idea on a sheet of
paper, with the postscript: "Six thousand feet beyond man and time."
That day I happened to be wandering through the woods alongside of the
Lake of Silvaplana, and I halted not far from Surlei, beside a huge
rock that towered aloft like a pyramid. It was then that the thought
struck me. Looking back now, I find that exactly two months before
this inspiration I had an omen of its coming in the form of a sudden
and decisive change in my tastes--more particularly in music. The
whole of _Zarathustra_ might perhaps be classified under the rubric
music. At all events, the essential condition of its production was
a second birth within me of the art of hearing. In Recoaro, a small
mountain resort near Vicenza, where I spent the spring of 1881, I and
my friend and maestro, Peter Gast--who was also one who had been born
again, discovered that the phœnix music hovered over us, in lighter
and brighter plumage than it had ever worn before. If, therefore, I
now calculate from that day forward the sudden production of the book,
under the most unlikely circumstances, in February 1883,--the last
part, out of which I quoted a few lines in my preface, was written
precisely in the hallowed hour when Richard Wagner gave up the ghost
in Venice,--I come to the conclusion that the period of gestation
covered eighteen months. This period of exactly eighteen months,
might suggest, at least to Buddhists, that I am in reality a female
elephant The interval was devoted to the _Gaya Scienza,_ which contains
hundreds of indications of the proximity of something unparalleled;
for, after all, it shows the beginning of _Zarathustra,_ since it
presents _Zarathustra's_ fundamental thought in the last aphorism
but one of the fourth book. To this interval also belongs that _Hymn
to Life_ (for a mixed choir and orchestra), the score of which was
published in Leipzig two years ago by E. W. Fritsch, and which gave
perhaps no slight indication of my spiritual state during this year,
in which the essentially yea-saying pathos, which I call the tragic
pathos, completely filled me heart and limb. One day people will sing
it to my memory. The text, let it be well understood, as there is
some misunderstanding abroad on this point, is not by me; it was the
astounding inspiration of a young Russian lady, Miss Lou von Salome,
with whom I was then on friendly terms. He who is in any way able
to make some sense of the last words of the poem, will divine why I
preferred and admired it: there is greatness in them. Pain is not
regarded as an objection to existence: "And if thou hast no bliss now
left to crown me--Lead on! Thou hast thy Sorrow still."

Maybe that my music is also great in this passage. (The last note of
the oboe, by the bye, is C sharp, not C. The latter is a misprint.)
During the following winter, I was living on that charmingly peaceful
Gulf of Rapallo, not far from Genoa, which cuts inland between Chiavari
and Cape Porto Fino. My health was not very good; the winter was cold
and exceptionally rainy; and the small _albergo_ in which I lived
was so close to the water that at night my sleep was disturbed if
the sea was rough. These circumstances were surely the very reverse
of favourable; and yet, in spite of it all, and as if in proof of my
belief that everything decisive comes to life in defiance of every
obstacle, it was precisely during this winter and in the midst of these
unfavourable circumstances that my _Zarathustra_ originated. In the
morning I used to start out in a southerly direction up the glorious
road to Zoagli, which rises up through a forest of pines and gives
one a view far out to sea. In the afternoon, or as often as my health
allowed, I walked round the whole bay from Santa Margherita to beyond
Porto Fino. This spot affected me all the more deeply because it was
so dearly loved by the Emperor Frederick III. In the autumn of 1886 I
chanced to be there again when he was revisiting this small forgotten
world of happiness for the last time. It was on these two roads that
all _Zarathustra_ came to me, above all, Zarathustra himself as a
type--I ought rather to say that it was on these walks that _he waylaid
me_.
""")

' I first conceived the idea of the Eternal Recurrence in August 1881 while wandering through the woods near Lake Silvaplana. The concept was born from a sudden change in my musical tastes, and it took eighteen months for the idea to fully develop into the book, _Zarathustra_. The majority of the work was written during a cold and rainy winter in Rapallo, where I often walked along the beautiful roads to Zoagli and Porto Fino.'

### Mixtral chunk fn

In [ ]:
# Bkp
"""
### Example response ARRAY with SENTENCE NUMBERS:

```
{{
    "number_of_splits": 3,
    "sentence_number_array": [1, 4, 7],
    "chunk_topic_array": ["Cat", "Dog", "Soccer"]
}}

### Example input split paragraph SENTENCES:

1. There once was a cat, his name was Midnight!
2. It was a black cat and he was up to no good.
3. He often climbed trees.
4. Another animal was a dog named Gcode.
5. This dog was always listening to gangster rap.
6. Which of course is strange behavior for any animal.
7. Lastly I want to talk about soccer.
8. Why would anyone watch that?
9. It's a sport made for really dumb people.

```
"""

In [ ]:
class ChunkData(BaseModel):
    number_of_splits: int = Field(description="The number of chunks")
    sentence_number_array: List[int] = Field(description="The sentence numbers where new chunk start based on topic")
    chunk_topic_array: List[str] = Field(description="The 1 word topic per chunk")

parser = JsonOutputParser(pydantic_object=ChunkData)

chunk_prompt_txt = """
### Context and question request:
- You are a professional paragraph chunker
- You will recieve a paragraph from a book which is split into enumerated sentences
- You will split it into {n_sections} sections (a.k.a. chunks), by responding with the cutoff points (array of integers) where the new section starts
- THERE SHOULD BE {n_sections} so there should be {n_sections} entries in the array!
- You should split the sections based on TOPICS WITHIN THE PARAGRAPH
- The first section for "sentence_number_array" always starts with 1, so the array will also always start with [1, ...
- Add an array to the output "chunk_topic_array" with the topic per chunk
- The "number_of_splits" part of the JSON should just contain the number of chunks, so in this case {n_sections}

### PARAGRAPH IN ENUMERATED SENTENCES:

{sentences}

### RESPONSE ARRAY with {n_sections} entries:
"""

chunk_prompt = ChatPromptTemplate.from_template(chunk_prompt_txt)

chunk_prompt = PromptTemplate(
    template=chunk_prompt_txt,
    input_variables=['paragraph', 'n_sections'],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# chunk_chain = chunk_prompt | openai_llm_4 | parser
# chunk_chain = chunk_prompt | openrouter_llm_4
chunk_chain = chunk_prompt | openai_llm_3 | parser
# chunk_chain = chunk_prompt | openrouter_llm_3
# chunk_chain = chunk_prompt | openrouter_llm | parser
# chunk_chain = chunk_prompt | openrouter_llm

def chunk_paragraph(paragraph, n_sections):
    # New idea: sentence splitter, sentence indexing and then let it choose the cutoff sentences
    sentences = paragraph.split('. ')
    enumerated_sentences = '\n'.join(f"{i+1}. {sentence}" for i, sentence in enumerate(sentences))
    print(enumerated_sentences)

    response = chunk_chain.invoke({'sentences': sentences, 'n_sections': n_sections})
    print(response)
    print('⏩ number_of_splits', response['number_of_splits'])
    print('⏩ sentence_number_array', response['sentence_number_array'])
    print('⏩ chunk_topic_array', response['chunk_topic_array'])
    sentence_number_array = response['sentence_number_array']

    chunks = []
    for i in range(len(sentence_number_array)):
        # Determine the start and end indices for each chunk
        start_index = sentence_number_array[i] - 1
        end_index = sentence_number_array[i+1] - 1 if i+1 < len(sentence_number_array) else len(sentences)
        # Join sentences for the current chunk and append
        chunks.append('. '.join(sentences[start_index:end_index]))
    return chunks

    print(chunks)

    # return response.content
    return response

In [ ]:
# Test with clear distinction
paragraph = """
There once was a cat, his name was Midnight! It was a black cat and he was up to no good. He often climbed trees. Another animal was a dog named Gcode. This dog was always listening to gangster rap. Which of course is strange behavior for any animal. Lastly I want to talk about soccer. Why would anyone watch that? It's a sport made for really dumb people.
"""

test_chunks = chunk_paragraph(paragraph, 3)
test_chunks

1. 
There once was a cat, his name was Midnight! It was a black cat and he was up to no good
2. He often climbed trees
3. Another animal was a dog named Gcode
4. This dog was always listening to gangster rap
5. Which of course is strange behavior for any animal
6. Lastly I want to talk about soccer
7. Why would anyone watch that? It's a sport made for really dumb people.

{'sentence_number_array': [1, 3, 6], 'chunk_topic_array': ['Midnight the cat', 'Gcode the dog', 'Soccer'], 'number_of_splits': 3}
⏩ number_of_splits 3
⏩ sentence_number_array [1, 3, 6]
⏩ chunk_topic_array ['Midnight the cat', 'Gcode the dog', 'Soccer']


['\nThere once was a cat, his name was Midnight! It was a black cat and he was up to no good. He often climbed trees',
 'Another animal was a dog named Gcode. This dog was always listening to gangster rap. Which of course is strange behavior for any animal',
 "Lastly I want to talk about soccer. Why would anyone watch that? It's a sport made for really dumb people.\n"]

In [ ]:
# Test with clear distinction
paragraph = """
Accordingly, if we have perceived this much, that Euripides did not succeed in establishing the drama exclusively on the Apollonian, but that rather his non-Dionysian inclinations deviated into a naturalistic and inartistic tendency, we shall now be able to approach nearer to the character _æsthetic Socratism._ supreme law of which reads about as follows: "to be beautiful everything must be intelligible," as the parallel to the Socratic proposition, "only the knowing is one virtuous." With this canon in his hands Euripides measured all the separate elements of the drama, and rectified them according to his principle: the language, the characters, the dramaturgic structure, and the choric music. The poetic deficiency and retrogression, which we are so often wont to impute to Euripides in comparison with Sophoclean tragedy, is for the most part the product of this penetrating critical process, this daring intelligibility. The Euripidian _prologue_ may serve us as an example of the productivity of this, rationalistic method. Nothing could be more opposed to the technique of our stage than the prologue in the drama of Euripides. For a single person to appear at the outset of the play telling us who he is, what precedes the action, what has happened thus far, yea, what will happen in the course of the play, would be designated by a modern playwright as a wanton and unpardonable abandonment of the effect of suspense. Everything that is about to happen is known beforehand; who then cares to wait for it actually to happen?--considering, moreover, that here there is not by any means the exciting relation of a predicting dream to a reality taking place later on. Euripides speculated quite differently. The effect of tragedy never depended on epic suspense, on the fascinating uncertainty as to what is to happen now and afterwards: but rather on the great rhetoro-lyric scenes in which the passion and dialectics of the chief hero swelled to a broad and mighty stream. Everything was arranged for pathos, not for action: and whatever was not arranged for pathos was regarded as objectionable. But what interferes most with the hearer's pleasurable satisfaction in such scenes is a missing link, a gap in the texture of the previous history. So long as the spectator has to divine the meaning of this or that person, or the presuppositions of this or that conflict of inclinations and intentions, his complete absorption in the doings and sufferings of the chief persons is impossible, as is likewise breathless fellow-feeling and fellow-fearing. The Æschyleo-Sophoclean tragedy employed the most ingenious devices in the first scenes to place in the hands of the spectator as if by chance all the threads requisite for understanding the whole: a trait in which that noble artistry is approved, which as it were masks the _inevitably_ formal, and causes it to appear as something accidental. But nevertheless Euripides thought he observed that during these first scenes the spectator was in a strange state of anxiety to make out the problem of the previous history, so that the poetic beauties and pathos of the exposition were lost to him. Accordingly he placed the prologue even before the exposition, and put it in the mouth of a person who could be trusted: some deity had often as it were to guarantee the particulars of the tragedy to the public and remove every doubt as to the reality of the myth: as in the case of Descartes, who could only prove the reality of the empiric world by an appeal to the truthfulness of God and His inability to utter falsehood. Euripides makes use of the same divine truthfulness once more at the close of his drama, in order to ensure to the public the future of his heroes; this is the task of the notorious _deus ex machina._ Between the preliminary and the additional epic spectacle there is the dramatico-lyric present, the "drama" proper.
"""

print()
test_chunks = chunk_paragraph(paragraph, 5)
for chunk in test_chunks:
    print(chunk)
    print()


1. 
Accordingly, if we have perceived this much, that Euripides did not succeed in establishing the drama exclusively on the Apollonian, but that rather his non-Dionysian inclinations deviated into a naturalistic and inartistic tendency, we shall now be able to approach nearer to the character _æsthetic Socratism._ supreme law of which reads about as follows: "to be beautiful everything must be intelligible," as the parallel to the Socratic proposition, "only the knowing is one virtuous." With this canon in his hands Euripides measured all the separate elements of the drama, and rectified them according to his principle: the language, the characters, the dramaturgic structure, and the choric music
2. The poetic deficiency and retrogression, which we are so often wont to impute to Euripides in comparison with Sophoclean tragedy, is for the most part the product of this penetrating critical process, this daring intelligibility
3. The Euripidian _prologue_ may serve us as an example of t

# 📕 Prepare the books

In [ ]:
corpus_df_flat = pd.read_csv('https://storage.googleapis.com/nietzsche_db/NietzscheDB.csv')
corpus_df_flat

,content,chapter,subchapter,book_id
0,Whatever may lie at the bottom of this doubtfu...,ATTEMPT AT A SELF-CRITICISM,I.,birth_of_tragedy
1,We can thus guess where the great note of inte...,ATTEMPT AT A SELF-CRITICISM,I.,birth_of_tragedy
2,"What I then laid hands on, something terrible ...",ATTEMPT AT A SELF-CRITICISM,2.,birth_of_tragedy
3,"I say again, to-day it is an impossible book t...",ATTEMPT AT A SELF-CRITICISM,3.,birth_of_tragedy
4,"Ay, what is Dionysian?--In this book may be fo...",ATTEMPT AT A SELF-CRITICISM,4.,birth_of_tragedy
...,...,...,...,...
3079,But I have chosen the title of Immoral is t as...,WHY I AM A FATALITY,6,ecce_homo
3080,"Have you understood me? That which defines me,...",WHY I AM A FATALITY,7,ecce_homo
3081,Have you understood me? I have not uttered a s...,WHY I AM A FATALITY,8,ecce_homo
3082,9,WHY I AM A FATALITY,8,ecce_homo


In [ ]:
corpus_df = corpus_df_flat.groupby(['chapter', 'subchapter', 'book_id'])['content'].agg('\n\n'.join).reset_index()
corpus_df = corpus_df.rename(columns={'content': 'full_subchapter'})
corpus_df

,chapter,subchapter,book_id,full_subchapter
0,ATTEMPT AT A SELF-CRITICISM,2.,birth_of_tragedy,"What I then laid hands on, something terrible ..."
1,ATTEMPT AT A SELF-CRITICISM,3.,birth_of_tragedy,"I say again, to-day it is an impossible book t..."
2,ATTEMPT AT A SELF-CRITICISM,4.,birth_of_tragedy,"Ay, what is Dionysian?--In this book may be fo..."
3,ATTEMPT AT A SELF-CRITICISM,5.,birth_of_tragedy,"Already in the foreword to Richard Wagner, art..."
4,ATTEMPT AT A SELF-CRITICISM,6.,birth_of_tragedy,You see which problem I ventured to touch upon...
...,...,...,...,...
1606,WHY I WRITE SUCH EXCELLENT BOOKS,1,ecce_homo,"I am one thing, my creations are another. Here..."
1607,WHY I WRITE SUCH EXCELLENT BOOKS,2,ecce_homo,This was said for the benefit of Germans: for ...
1608,WHY I WRITE SUCH EXCELLENT BOOKS,3,ecce_homo,I am to a great extent aware of my privileges ...
1609,WHY I WRITE SUCH EXCELLENT BOOKS,4,ecce_homo,I will now pass just one or two general remark...


# ✍🏻 Summarize!

In [ ]:
corpus_df

,chapter,subchapter,book_id,full_subchapter,content_short
0,ATTEMPT AT A SELF-CRITICISM,2.,birth_of_tragedy,"What I then laid hands on, something terrible ...",None
1,ATTEMPT AT A SELF-CRITICISM,3.,birth_of_tragedy,"I say again, to-day it is an impossible book t...",None
2,ATTEMPT AT A SELF-CRITICISM,4.,birth_of_tragedy,"Ay, what is Dionysian?--In this book may be fo...",None
3,ATTEMPT AT A SELF-CRITICISM,5.,birth_of_tragedy,"Already in the foreword to Richard Wagner, art...",None
4,ATTEMPT AT A SELF-CRITICISM,6.,birth_of_tragedy,You see which problem I ventured to touch upon...,None
...,...,...,...,...,...
1606,WHY I WRITE SUCH EXCELLENT BOOKS,1,ecce_homo,"I am one thing, my creations are another. Here...",None
1607,WHY I WRITE SUCH EXCELLENT BOOKS,2,ecce_homo,This was said for the benefit of Germans: for ...,None
1608,WHY I WRITE SUCH EXCELLENT BOOKS,3,ecce_homo,I am to a great extent aware of my privileges ...,None
1609,WHY I WRITE SUCH EXCELLENT BOOKS,4,ecce_homo,I will now pass just one or two general remark...,None


In [ ]:
import time
from IPython.display import clear_output

corpus_df['chapter_short'] = None

for index, row in corpus_df.iterrows():
    if len(row['full_subchapter']) < 300:
        corpus_df.at[index, 'chapter_short'] = row['full_subchapter']
    else:
        try:
            summary = summarize_chapter(row['full_subchapter'])
            time.sleep(0.1)  # Wait 0.1 sec before the next iteration
            corpus_df.at[index, 'chapter_short'] = summary
        except Exception as e:
            print(f"Error summarizing content at index {index}: {e}")
            # Optionally, save the current state of corpus_df to a file for recovery
            corpus_df.to_csv('corpus_progress.csv', index=False)
            # No need to exit; we can just note the failure and move on
    # Update progress
    clear_output(wait=True)
    print(f"Processing index {index}/{len(corpus_df)-1}")

corpus_df.to_csv('corpus_final.csv', index=False)
print("Processing complete")
corpus_df


# There are some empty values still❗

Processing index 1610/1610
Processing complete


,chapter,subchapter,book_id,full_subchapter,content_short,chapter_short
0,ATTEMPT AT A SELF-CRITICISM,2.,birth_of_tragedy,"What I then laid hands on, something terrible ...",None,"In this chapter, the author reflects on their..."
1,ATTEMPT AT A SELF-CRITICISM,3.,birth_of_tragedy,"I say again, to-day it is an impossible book t...",None,This chapter discusses the difficulty in unde...
2,ATTEMPT AT A SELF-CRITICISM,4.,birth_of_tragedy,"Ay, what is Dionysian?--In this book may be fo...",None,"In this chapter, the author explores the conc..."
3,ATTEMPT AT A SELF-CRITICISM,5.,birth_of_tragedy,"Already in the foreword to Richard Wagner, art...",None,"In this chapter, the author presents a metaph..."
4,ATTEMPT AT A SELF-CRITICISM,6.,birth_of_tragedy,You see which problem I ventured to touch upon...,None,"In this chapter, the author reflects on their..."
...,...,...,...,...,...,...
1606,WHY I WRITE SUCH EXCELLENT BOOKS,1,ecce_homo,"I am one thing, my creations are another. Here...",None,I discuss the misunderstandings and criticism...
1607,WHY I WRITE SUCH EXCELLENT BOOKS,2,ecce_homo,This was said for the benefit of Germans: for ...,None,I am a philosopher who has been discovered in...
1608,WHY I WRITE SUCH EXCELLENT BOOKS,3,ecce_homo,I am to a great extent aware of my privileges ...,None,I am a writer who is aware of my privileges a...
1609,WHY I WRITE SUCH EXCELLENT BOOKS,4,ecce_homo,I will now pass just one or two general remark...,None,"In this chapter, the author discusses their u..."


In [ ]:
import time
from IPython.display import clear_output

# Fill in empty values
corpus_df = pd.read_csv('corpus_final.csv')

empty_count = corpus_df['chapter_short'].isna().sum() + (corpus_df['chapter_short'] == '').sum()

for index, row in corpus_df.iterrows():
    # Check if 'chapter_short' is empty
    if pd.isna(row['chapter_short']) or row['chapter_short'] == '':
        if len(row['full_subchapter']) < 300:
            corpus_df.at[index, 'chapter_short'] = row['full_subchapter']
        else:
            try:
                summary = summarize_chapter(row['full_subchapter'])
                time.sleep(0.1)  # Wait 0.1 sec before the next iteration
                corpus_df.at[index, 'chapter_short'] = summary
            except Exception as e:
                print(f"Error summarizing content at index {index}: {e}")
                # Optionally, save the current state of corpus_df to a file for recovery
                corpus_df.to_csv('corpus_progress.csv', index=False)
                # No need to exit; we can just note the failure and move on
    # Update progress
    clear_output(wait=True)
    print(f"Processing index {index}/{len(corpus_df)-1}")

corpus_df.to_csv('corpus_final2.csv', index=False)
print("Processing complete")
empty_count = corpus_df['chapter_short'].isna().sum() + (corpus_df['chapter_short'] == '').sum()

Processing index 1610/1610
Processing complete


0

# 🍪 Semantic chunking!
- [ ] What max chunk size? -> Let's start with 250
- [ ] Columns: `chunk`, `chunk_chapter_index`, `keywords`

In [ ]:
!pip install -q tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.9 MB/s eta 0:00:00


In [ ]:
import tiktoken

def num_tokens_from_string(string, model = 'gpt-4'):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = len(encoding.encode(string))
    return num_tokens

- text-embedding-3-small | $0.00002 per 1K tokens

- text-embedding-3-large | $0.00013 per 1K tokens


In [ ]:
# Print some statistics
MAX_CHUNK_TOKENS = 250
ONE_K_TOKEN_PRICE_SMALL = 0.00002
ONE_K_TOKEN_PRICE_LARGE = 0.00013

print('⏩ Number of chunks', len(corpus_df))

# ❗ Rename
def chunker(chunk):
    if (num_tokens_from_string(chunk) > MAX_CHUNK_TOKENS):
        return True
    return False

# Loop through each chunk in bot_df and count the number of big chunks
corpus_df['chunksize'] = corpus_df['content'].apply(num_tokens_from_string)

# print(corpus_df['chunksize'] > MAX_CHUNK_TOKENS) # Fix this, count the > MAX_CHUNK_TOKENS chunks
print('⏩ Large chunks', (corpus_df['chunksize'] > MAX_CHUNK_TOKENS).sum())

total_tokens = corpus_df['chunksize'].sum()
print('⏩ Total tokens', total_tokens)
print('⏩ Token cost small', (total_tokens / 1000) * ONE_K_TOKEN_PRICE_SMALL)
print('⏩ Token cost large', (total_tokens / 1000) * ONE_K_TOKEN_PRICE_LARGE)

⏩ Number of chunks 1085
⏩ Large chunks 324
⏩ Total tokens 224006
⏩ Token cost small 0.004480120000000001
⏩ Token cost large 0.02912078


### Naive test chunker

In [ ]:
"""
If a chunk is > 300
Divide the chunk by 300 and round up
So if a chunk is size 654, it must be converted into 3 chunks

chunksize / 300

"""

chunk_df = corpus_df.copy()

def chunker(row):
    chunk = row['content']
    n_sub_chunks = int(np.ceil(num_tokens_from_string(chunk) / MAX_CHUNK_TOKENS))

    chunk_size = len(chunk) // n_sub_chunks + (len(chunk) % n_sub_chunks > 0)
    sub_chunks = [chunk[i:i + chunk_size] for i in range(0, len(chunk), chunk_size)]

    df = pd.DataFrame({
        'sub_chunks': sub_chunks,
        'sub_chunk_summary': ['the unconditional acceptance of all life and experience'] * n_sub_chunks,
        'fake_keywords': [['morality', 'amor fati', 'suffering', 'illness']] * n_sub_chunks
    })

    # Copying other columns from the original row to the new DataFrame
    for col in row.index:
        if col not in ['content', 'sub_chunks', 'sub_chunk_summary', 'fake_keywords']:
            df[col] = row[col]

    return df

sub_chunk_df = pd.concat([chunker(row) for _, row in chunk_df.iterrows()], ignore_index=True)
sub_chunk_df

,sub_chunks,sub_chunk_summary,fake_keywords,chapter,subchapter,chapter_index,subchapter_index,book_id,chunksize
0,Whatever may lie at the bottom of this doubtfu...,the unconditional acceptance of all life and e...,"[morality, amor fati, suffering, illness]",ATTEMPT AT A SELF-CRITICISM,I.,1,1,birth_of_tragedy,355
1,er: and he found himself under the walls of Me...,the unconditional acceptance of all life and e...,"[morality, amor fati, suffering, illness]",ATTEMPT AT A SELF-CRITICISM,I.,1,1,birth_of_tragedy,355
2,We can thus guess where the great note of inte...,the unconditional acceptance of all life and e...,"[morality, amor fati, suffering, illness]",ATTEMPT AT A SELF-CRITICISM,I.,1,1,birth_of_tragedy,436
3,"ch wasborn thereof, tragedy?--And again: that ...",the unconditional acceptance of all life and e...,"[morality, amor fati, suffering, illness]",ATTEMPT AT A SELF-CRITICISM,I.,1,1,birth_of_tragedy,436
4,"What I then laid hands on, something terrible ...",the unconditional acceptance of all life and e...,"[morality, amor fati, suffering, illness]",ATTEMPT AT A SELF-CRITICISM,2.,1,2,birth_of_tragedy,484
...,...,...,...,...,...,...,...,...,...
1509,"LIFE SHALL COMFORT US.--If, like the thinker, ...",the unconditional acceptance of all life and e...,"[morality, amor fati, suffering, illness]",BOOK V.,572.,6,151,daybreak,60
1510,CASTING ONE'S SKIN.--The snake that cannot cas...,the unconditional acceptance of all life and e...,"[morality, amor fati, suffering, illness]",BOOK V.,573.,6,152,daybreak,38
1511,NEVER FORGET!--The higher we soar the smaller ...,the unconditional acceptance of all life and e...,"[morality, amor fati, suffering, illness]",BOOK V.,574.,6,153,daybreak,19
1512,WE AERONAUTS OF THE INTELLECT.--All those dari...,the unconditional acceptance of all life and e...,"[morality, amor fati, suffering, illness]",BOOK V.,575.,6,154,daybreak,243


### Mixtral chunker (failed experiment)

In [ ]:
!pip install -q semantic-text-splitter -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 17.6 MB/s eta 0:00:00


In [ ]:
from semantic_text_splitter import HuggingFaceTextSplitter
from tokenizers import Tokenizer

In [ ]:
tokenizer = Tokenizer.from_pretrained("bert-base-uncased")

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
content = """
Accordingly, if we have perceived this much, that Euripides did not
succeed in establishing the drama exclusively on the Apollonian, but
that rather his non-Dionysian inclinations deviated into a naturalistic
and inartistic tendency, we shall now be able to approach nearer to
the character _æsthetic Socratism._ supreme law of which reads about
as follows: "to be beautiful everything must be intelligible," as
the parallel to the Socratic proposition, "only the knowing is one
virtuous." With this canon in his hands Euripides measured all the
separate elements of the drama, and rectified them according to his
principle: the language, the characters, the dramaturgic structure, and
the choric music. The poetic deficiency and retrogression, which we
are so often wont to impute to Euripides in comparison with Sophoclean
tragedy, is for the most part the product of this penetrating critical
process, this daring intelligibility. The Euripidian _prologue_ may
serve us as an example of the productivity of this, rationalistic
method. Nothing could be more opposed to the technique of our stage
than the prologue in the drama of Euripides. For a single person to
appear at the outset of the play telling us who he is, what precedes
the action, what has happened thus far, yea, what will happen in
the course of the play, would be designated by a modern playwright
as a wanton and unpardonable abandonment of the effect of suspense.
Everything that is about to happen is known beforehand; who then
cares to wait for it actually to happen?--considering, moreover, that
here there is not by any means the exciting relation of a predicting
dream to a reality taking place later on. Euripides speculated quite
differently. The effect of tragedy never depended on epic suspense, on
the fascinating uncertainty as to what is to happen now and afterwards:
but rather on the great rhetoro-lyric scenes in which the passion and
dialectics of the chief hero swelled to a broad and mighty stream.
Everything was arranged for pathos, not for action: and whatever
was not arranged for pathos was regarded as objectionable. But what
interferes most with the hearer's pleasurable satisfaction in such
scenes is a missing link, a gap in the texture of the previous history.
So long as the spectator has to divine the meaning of this or that
person, or the presuppositions of this or that conflict of inclinations
and intentions, his complete absorption in the doings and sufferings
of the chief persons is impossible, as is likewise breathless
fellow-feeling and fellow-fearing. The Æschyleo-Sophoclean tragedy
employed the most ingenious devices in the first scenes to place in
the hands of the spectator as if by chance all the threads requisite
for understanding the whole: a trait in which that noble artistry is
approved, which as it were masks the _inevitably_ formal, and causes
it to appear as something accidental. But nevertheless Euripides
thought he observed that during these first scenes the spectator was
in a strange state of anxiety to make out the problem of the previous
history, so that the poetic beauties and pathos of the exposition
were lost to him. Accordingly he placed the prologue even before the
exposition, and put it in the mouth of a person who could be trusted:
some deity had often as it were to guarantee the particulars of the
tragedy to the public and remove every doubt as to the reality of the
myth: as in the case of Descartes, who could only prove the reality
of the empiric world by an appeal to the truthfulness of God and His
inability to utter falsehood. Euripides makes use of the same divine
truthfulness once more at the close of his drama, in order to ensure to
the public the future of his heroes; this is the task of the notorious
_deus ex machina._ Between the preliminary and the additional epic
spectacle there is the dramatico-lyric present, the "drama" proper.
"""

In [ ]:
MIN_TOKENS = 50
MAX_TOKENS = 1000

chunks_with_model = splitter.chunks(content, chunk_capacity=(MIN_TOKENS, MAX_TOKENS))
for i, chunk in enumerate(chunks_with_model):
    print(f"CHUNK {i+1}: ", chunk)

CHUNK 1:  
Accordingly, if we have perceived this much, that Euripides did not
succeed in establishing the drama exclusively on the Apollonian, but
that rather his non-Dionysian inclinations deviated into a naturalistic
and inartistic tendency, we shall now be able to approach nearer to

CHUNK 2:  the character _æsthetic Socratism._ supreme law of which reads about
as follows: "to be beautiful everything must be intelligible," as
the parallel to the Socratic proposition, "only the knowing is one
virtuous." With this canon in his hands Euripides measured all the

CHUNK 3:  separate elements of the drama, and rectified them according to his
principle: the language, the characters, the dramaturgic structure, and
the choric music. The poetic deficiency and retrogression, which we
are so often wont to impute to Euripides in comparison with Sophoclean

CHUNK 4:  tragedy, is for the most part the product of this penetrating critical
process, this daring intelligibility. The Euripidian _prolog

### Cleanup

In [ ]:
# Some modifications, probably better placed somewhere else in the code
sub_chunk_df = sub_chunk_df.drop(columns=['chunksize'])
sub_chunk_df = sub_chunk_df.drop(columns=['fake_keywords']) # ❗ Drop the keyword function for now, chromadb doesn't like lists
sub_chunk_df['chunk_index'] = sub_chunk_df.groupby('subchapter_index').cumcount()
sub_chunk_df

,sub_chunks,sub_chunk_summary,chapter,subchapter,chapter_index,subchapter_index,book_id,chunk_index
0,Whatever may lie at the bottom of this doubtfu...,the unconditional acceptance of all life and e...,ATTEMPT AT A SELF-CRITICISM,I.,1,1,birth_of_tragedy,0
1,er: and he found himself under the walls of Me...,the unconditional acceptance of all life and e...,ATTEMPT AT A SELF-CRITICISM,I.,1,1,birth_of_tragedy,1
2,We can thus guess where the great note of inte...,the unconditional acceptance of all life and e...,ATTEMPT AT A SELF-CRITICISM,I.,1,1,birth_of_tragedy,2
3,"ch wasborn thereof, tragedy?--And again: that ...",the unconditional acceptance of all life and e...,ATTEMPT AT A SELF-CRITICISM,I.,1,1,birth_of_tragedy,3
4,"What I then laid hands on, something terrible ...",the unconditional acceptance of all life and e...,ATTEMPT AT A SELF-CRITICISM,2.,1,2,birth_of_tragedy,0
...,...,...,...,...,...,...,...,...
1509,"LIFE SHALL COMFORT US.--If, like the thinker, ...",the unconditional acceptance of all life and e...,BOOK V.,572.,6,151,daybreak,1
1510,CASTING ONE'S SKIN.--The snake that cannot cas...,the unconditional acceptance of all life and e...,BOOK V.,573.,6,152,daybreak,1
1511,NEVER FORGET!--The higher we soar the smaller ...,the unconditional acceptance of all life and e...,BOOK V.,574.,6,153,daybreak,1
1512,WE AERONAUTS OF THE INTELLECT.--All those dari...,the unconditional acceptance of all life and e...,BOOK V.,575.,6,154,daybreak,1


# Global data per book utility

Should be temprary